#### CSC 180 Intelligent Systems 

#### Dr. Haiquan Chen, Dept of Computer Scicence

#### California State University, Sacramento




## Lab 2: Tensorflow Introduction



### Helpful Functions for Tensorflow (little gems)

The following functions will be used with TensorFlow to help preprocess the data.  They allow you to build the feature vector for a neural network. 

* Predictors/Inputs 
    * Fill any missing inputs with the median for that column.  Use **missing_median**.
    * Encode textual/categorical values with **encode_text_dummy**.
    * Encode numeric values with **encode_numeric_zscore**.
* Output
    * Discard rows with missing outputs.
    * Encode textual/categorical values with **encode_text_index**.
    * Do not encode output numeric values.
* Produce final feature vectors (x) and expected output (y) with **to_xy**.

In [1]:
from collections.abc import Sequence
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import os


# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)



# Encode text values to indexes(i.e. [1],[2],[3] for red,green,blue).
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_


# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd


# Convert all missing values in the specified column to the median
def missing_median(df, name):
    med = df[name].median()
    df[name] = df[name].fillna(med)


# Convert all missing values in the specified column to the default
def missing_default(df, name, default_value):
    df[name] = df[name].fillna(default_value)


# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column. 
    target_type = df[target].dtypes
    target_type = target_type[0] if isinstance(target_type, Sequence) else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    else:
        # Regression
        return df[result].values.astype(np.float32), df[target].values.astype(np.float32)

# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Regression chart.
def chart_regression(pred,y,sort=True):
    t = pd.DataFrame({'pred' : pred, 'y' : y.flatten()})
    if sort:
        t.sort_values(by=['y'],inplace=True)
    a = plt.plot(t['y'].tolist(),label='expected')
    b = plt.plot(t['pred'].tolist(),label='prediction')
    plt.ylabel('output')
    plt.legend()
    plt.show()

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean()) >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)


# Encode a column to a range between normalized_low and normalized_high.
def encode_numeric_range(df, name, normalized_low=-1, normalized_high=1,
                         data_low=None, data_high=None):
    if data_low is None:
        data_low = min(df[name])
        data_high = max(df[name])

    df[name] = ((df[name] - data_low) / (data_high - data_low)) \
               * (normalized_high - normalized_low) + normalized_low


# Classification or Regression

Neural networks can function in *** classification or regression***:

* **Regression** - You expect a number as your neural network's prediction.
* **Classification** - You expect a class/category as your neural network's prediction.

Regression networks always have a single output neuron.  Classification neural networks have an output neuron for each class. 

These neurons are grouped into layers:

* **Input Layer** - The input layer accepts feature vectors from the dataset.  Input layers usually have a bias neuron.
* **Output Layer** - The output from the neural network.  The output layer does not have a bias neuron.
* **Hidden Layers** - Layers that occur between the input and output layers.  Each hidden layer will usually have a bias neuron.


# Example of TensorFlow Regression: MPG Prediction

This example shows how to encode the MPG dataset for regression.  Notice that:

* Input has both numeric and categorical
* Input has missing values

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./data/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

df[0:5]

2024-09-16 11:14:43.205680: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [4]:
cars = df['name']

df.drop('name', axis = 1, inplace=True)

missing_median(df, 'horsepower')

# one hot encoding on categorial feature (origin)
encode_text_dummy(df, 'origin')

x,y = to_xy(df,"mpg")

In [5]:
x.shape

(398, 9)

In [6]:
y.shape

(398,)

In [7]:
x

array([[  8., 307., 130., ...,   1.,   0.,   0.],
       [  8., 350., 165., ...,   1.,   0.,   0.],
       [  8., 318., 150., ...,   1.,   0.,   0.],
       ...,
       [  4., 135.,  84., ...,   1.,   0.,   0.],
       [  4., 120.,  79., ...,   1.,   0.,   0.],
       [  4., 119.,  82., ...,   1.,   0.,   0.]], dtype=float32)

In [8]:
y

array([18. , 15. , 18. , 16. , 17. , 15. , 14. , 14. , 14. , 15. , 15. ,
       14. , 15. , 14. , 24. , 22. , 18. , 21. , 27. , 26. , 25. , 24. ,
       25. , 26. , 21. , 10. , 10. , 11. ,  9. , 27. , 28. , 25. , 25. ,
       19. , 16. , 17. , 19. , 18. , 14. , 14. , 14. , 14. , 12. , 13. ,
       13. , 18. , 22. , 19. , 18. , 23. , 28. , 30. , 30. , 31. , 35. ,
       27. , 26. , 24. , 25. , 23. , 20. , 21. , 13. , 14. , 15. , 14. ,
       17. , 11. , 13. , 12. , 13. , 19. , 15. , 13. , 13. , 14. , 18. ,
       22. , 21. , 26. , 22. , 28. , 23. , 28. , 27. , 13. , 14. , 13. ,
       14. , 15. , 12. , 13. , 13. , 14. , 13. , 12. , 13. , 18. , 16. ,
       18. , 18. , 23. , 26. , 11. , 12. , 13. , 12. , 18. , 20. , 21. ,
       22. , 18. , 19. , 21. , 26. , 15. , 16. , 29. , 24. , 20. , 19. ,
       15. , 24. , 20. , 11. , 20. , 21. , 19. , 15. , 31. , 26. , 32. ,
       25. , 16. , 16. , 18. , 16. , 13. , 14. , 14. , 14. , 29. , 26. ,
       26. , 31. , 32. , 28. , 24. , 26. , 24. , 26

In [9]:
# Add all layers sequentially
model = Sequential()

# 25 neurons in the first layer, i ** 
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1     
#  why input_dim=x.shape[1]?  Row number = 9


model.add(Dense(10, activation='relu')) # Hidden 2

# Do not set any activation function
model.add(Dense(1)) # Output

# we want to minimize error between the true value and the predicted value
# optimizer is for gradient decent
model.compile(loss='mean_squared_error', optimizer='adam')

# train and test on the entire dataset
# epoch goes through the same dataset 100 times
model.fit(x,y,verbose=2,epochs=100)    # Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch.


# start with larger than input features, then half the neurons (common practice)
# Output for regression should have only 1 neuron


Epoch 1/100
13/13 - 1s - loss: 26674.0547 - 879ms/epoch - 68ms/step
Epoch 2/100
13/13 - 0s - loss: 6427.9565 - 25ms/epoch - 2ms/step
Epoch 3/100
13/13 - 0s - loss: 1714.6616 - 29ms/epoch - 2ms/step
Epoch 4/100
13/13 - 0s - loss: 1351.8234 - 26ms/epoch - 2ms/step
Epoch 5/100
13/13 - 0s - loss: 1058.2068 - 26ms/epoch - 2ms/step
Epoch 6/100
13/13 - 0s - loss: 1033.9420 - 28ms/epoch - 2ms/step
Epoch 7/100
13/13 - 0s - loss: 965.9909 - 26ms/epoch - 2ms/step
Epoch 8/100
13/13 - 0s - loss: 929.6362 - 27ms/epoch - 2ms/step
Epoch 9/100
13/13 - 0s - loss: 896.9397 - 26ms/epoch - 2ms/step
Epoch 10/100
13/13 - 0s - loss: 862.9661 - 27ms/epoch - 2ms/step
Epoch 11/100
13/13 - 0s - loss: 833.5178 - 23ms/epoch - 2ms/step
Epoch 12/100
13/13 - 0s - loss: 797.6865 - 28ms/epoch - 2ms/step
Epoch 13/100
13/13 - 0s - loss: 762.2324 - 24ms/epoch - 2ms/step
Epoch 14/100
13/13 - 0s - loss: 727.6513 - 27ms/epoch - 2ms/step
Epoch 15/100
13/13 - 0s - loss: 693.8857 - 26ms/epoch - 2ms/step
Epoch 16/100
13/13 - 0s -

### Monitor the loss at each epoch

One line is produced for each training epoch.  You can eliminate this output by setting the verbose setting of the fit command:

* **verbose=0** - No progress output (use with Juputer if you do not want output)
* **verbose=1** - Display progress bar, does not work well with Jupyter
* **verbose=2** - Summary progress output (use with Jupyter if you want to know the loss at each epoch)


## Use Trained Model to Make Regression Prediction

Next we will perform actual predictions.  These predictions are assigned to the **pred** variable. These are all MPG predictions from the neural network.  

***Notice that the data to predict should be a 2D array!***  

***Notice that the prediction result is also a 2D array!*** 

Neural networks can return multiple values, so the result is always an array.  Here the neural network only returns 1 value per prediction (there are 398 cars, so 398 predictions).  However, a 2D array is needed because the neural network has the potential of returning more than one value. 


In [10]:
pred = model.predict(x)
print("Shape: {}".format(pred.shape))
print(pred)

13/13 [==============================] - 0s 1ms/step
Shape: (398, 1)
[[17.580786 ]
 [15.531317 ]
 [17.199163 ]
 [17.072882 ]
 [16.520437 ]
 [ 9.969839 ]
 [ 9.326712 ]
 [ 9.05982  ]
 [ 8.897985 ]
 [12.8342495]
 [17.493353 ]
 [14.246008 ]
 [17.080648 ]
 [23.210196 ]
 [23.410803 ]
 [22.229513 ]
 [22.799978 ]
 [25.796606 ]
 [25.290792 ]
 [33.78539  ]
 [21.54917  ]
 [22.143797 ]
 [23.97177  ]
 [22.559408 ]
 [24.257849 ]
 [ 4.7971244]
 [ 6.0351005]
 [ 5.1406913]
 [ 4.416616 ]
 [25.81072  ]
 [25.294455 ]
 [24.912687 ]
 [27.233503 ]
 [24.942968 ]
 [17.095541 ]
 [19.827383 ]
 [20.784063 ]
 [19.345602 ]
 [10.814413 ]
 [ 9.809149 ]
 [12.9575405]
 [11.779058 ]
 [ 3.413717 ]
 [ 7.351934 ]
 [ 2.862188 ]
 [22.487875 ]
 [26.707685 ]
 [20.051168 ]
 [21.969091 ]
 [24.231474 ]
 [26.126547 ]
 [28.937407 ]
 [26.37338  ]
 [32.085667 ]
 [33.067043 ]
 [32.700794 ]
 [29.992453 ]
 [25.73409  ]
 [26.509802 ]
 [31.573172 ]
 [26.504316 ]
 [26.080534 ]
 [10.639685 ]
 [11.452169 ]
 [12.160772 ]
 [13.46607  ]
 [15.27

We would like to see how good these predictions are.  We know what the correct MPG is for each car, so we can measure how close the neural network was.

In [11]:
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Final score (RMSE): {}".format(score))

Final score (RMSE): 3.9005420207977295


We can also print out the first 10 cars, with predictions and actual MPG.

In [20]:
# Sample predictions
for i in range(10):
    print("{}. Car name: {}, MPG: {}, predicted MPG: {}".format(i+1,cars[i],y[i],pred[i]))

1. Car name: chevrolet chevelle malibu, MPG: 18.0, predicted MPG: [17.05777]
2. Car name: buick skylark 320, MPG: 15.0, predicted MPG: [16.557625]
3. Car name: plymouth satellite, MPG: 18.0, predicted MPG: [17.76616]
4. Car name: amc rebel sst, MPG: 16.0, predicted MPG: [18.55326]
5. Car name: ford torino, MPG: 17.0, predicted MPG: [17.62445]
6. Car name: ford galaxie 500, MPG: 15.0, predicted MPG: [12.595313]
7. Car name: chevrolet impala, MPG: 14.0, predicted MPG: [12.585593]
8. Car name: plymouth fury iii, MPG: 14.0, predicted MPG: [12.858848]
9. Car name: pontiac catalina, MPG: 14.0, predicted MPG: [12.811301]
10. Car name: amc ambassador dpl, MPG: 15.0, predicted MPG: [15.092811]


# Example of TensorFlow Classification: Iris

This is a very simple example of how to perform the Iris classification using TensorFlow. The iris.csv file is used. 


In [15]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping


df=pd.read_csv("data/iris.csv",na_values=['NA','?'])

species = encode_text_index(df,"species")

x,y = to_xy(df,"species")

# output layer should be different (probability distribution)

In [16]:
x


array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [17]:
x.shape

(150, 4)

In [18]:
y  #  This is one-hot encoding.  Only one value is 1.0 (hot)

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0

In [19]:
y.shape

(150, 3)

In [20]:
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1

# up to the user
model.add(Dense(25, activation='relu')) # Hidden 2

# always the number of the output neurons should be equal to number of labels you want to predict
# probability distribution (softmax)
# MUST USE softmax as an activation function
model.add(Dense(y.shape[1], activation='softmax')) # Output

# gradient decent is different with each optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x,y,verbose=2,epochs=100)

Epoch 1/100
5/5 - 1s - loss: 1.1580 - 994ms/epoch - 199ms/step
Epoch 2/100
5/5 - 0s - loss: 1.0076 - 10ms/epoch - 2ms/step
Epoch 3/100
5/5 - 0s - loss: 0.9134 - 13ms/epoch - 3ms/step
Epoch 4/100
5/5 - 0s - loss: 0.8275 - 12ms/epoch - 2ms/step
Epoch 5/100
5/5 - 0s - loss: 0.7647 - 21ms/epoch - 4ms/step
Epoch 6/100
5/5 - 0s - loss: 0.7165 - 68ms/epoch - 14ms/step
Epoch 7/100
5/5 - 0s - loss: 0.6621 - 11ms/epoch - 2ms/step
Epoch 8/100
5/5 - 0s - loss: 0.6242 - 10ms/epoch - 2ms/step
Epoch 9/100
5/5 - 0s - loss: 0.5920 - 12ms/epoch - 2ms/step
Epoch 10/100
5/5 - 0s - loss: 0.5657 - 9ms/epoch - 2ms/step
Epoch 11/100
5/5 - 0s - loss: 0.5387 - 11ms/epoch - 2ms/step
Epoch 12/100
5/5 - 0s - loss: 0.5155 - 10ms/epoch - 2ms/step
Epoch 13/100
5/5 - 0s - loss: 0.4951 - 12ms/epoch - 2ms/step
Epoch 14/100
5/5 - 0s - loss: 0.4754 - 27ms/epoch - 5ms/step
Epoch 15/100
5/5 - 0s - loss: 0.4556 - 17ms/epoch - 3ms/step
Epoch 16/100
5/5 - 0s - loss: 0.4405 - 15ms/epoch - 3ms/step
Epoch 17/100
5/5 - 0s - loss: 

In [21]:
# Print out number of species found:
print(species)

['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']


Now that you have a neural network trained, we would like to be able to use it. There were 3 types of iris (Iris-setosa, Iris-versicolor, and Iris-virginica).  

In [22]:
pred = model.predict(x)
pred.shape

5/5 [==============================] - 0s 1ms/step


(150, 3)

In [29]:
pred

array([[9.99738395e-01, 2.61585577e-04, 4.15440411e-08],
       [9.98938859e-01, 1.06090936e-03, 2.42218334e-07],
       [9.99482214e-01, 5.17685781e-04, 1.51551518e-07],
       [9.98331606e-01, 1.66778360e-03, 6.52408687e-07],
       [9.99766767e-01, 2.33249040e-04, 4.27529621e-08],
       [9.99557316e-01, 4.42489749e-04, 6.65542927e-08],
       [9.99294877e-01, 7.04911537e-04, 2.91215429e-07],
       [9.99471247e-01, 5.28671080e-04, 1.06537570e-07],
       [9.97815728e-01, 2.18323339e-03, 1.12766020e-06],
       [9.99074340e-01, 9.25424392e-04, 1.94934245e-07],
       [9.99818146e-01, 1.81859912e-04, 1.86104234e-08],
       [9.99016404e-01, 9.83352074e-04, 2.89647119e-07],
       [9.99145389e-01, 8.54384853e-04, 2.00087811e-07],
       [9.99559700e-01, 4.40102449e-04, 1.88835202e-07],
       [9.99978662e-01, 2.13614258e-05, 8.90423957e-10],
       [9.99949694e-01, 5.02841722e-05, 3.88439192e-09],
       [9.99903440e-01, 9.65432191e-05, 1.12819523e-08],
       [9.99655843e-01, 3.44097

In [23]:
# print y.  
print(y)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0.

#### The column (pred) with the highest probability is  the prediction of the neural network.  

#### Use argmax function to find the index of the maximum prediction for each row.

In [31]:
# Usually the column (pred) with the highest prediction is considered to be the prediction of the neural network.  It is easy
# to convert the predictions to the expected iris species.  The argmax function finds the index of the maximum prediction
# for each row.

# returns the label with the highest value
predict_classes = np.argmax(pred,axis=1)

# returns the labl with the highest value
true_classes = np.argmax(y,axis=1)

print("Predictions: {}".format(predict_classes))
print("True: {}".format(true_classes))

Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 1
 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
True: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [32]:
# Of course it is very easy to turn these indexes back into iris species.  We just use the species list that we created earlier.
print(species[predict_classes[0:10]])

['Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa'
 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa' 'Iris-setosa']


In [33]:

#For all of the iris predictions, what percent were correct?  

correct = metrics.accuracy_score(true_classes, predict_classes)
print("Accuracy: {}".format(correct))

Accuracy: 0.98


The code below performs two ad hoc predictions.  

*** Remember x should be a 2D array! ***

In [25]:
# ad hoc prediction
sample_flower = np.array( [[5.0,3.0,4.0,2.0]], dtype=float)
pred = model.predict(sample_flower)
print(pred)

1/1 [==============================] - 0s 43ms/step
[[0.00157445 0.50584835 0.49257717]]


In [26]:
pred = np.argmax(pred, axis=1)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

Predict that [[5. 3. 4. 2.]] is: ['Iris-versicolor']


Notice that the argmax in the second prediction requires **axis=1**?  Since we have a 2D array now, we must specify which axis to take the argmax over.  The value **axis=1** specifies we want the max column index for each row.

In [27]:
# predict two sample flowers
sample_flower = np.array( [[5.0,3.0,4.0,2.0],[5.2,3.5,1.5,0.8]], dtype=float)
pred = model.predict(sample_flower)
print(pred)

1/1 [==============================] - 0s 28ms/step
[[1.5744535e-03 5.0584817e-01 4.9257734e-01]
 [9.9363059e-01 6.3692438e-03 1.5190689e-07]]


In [28]:
pred = np.argmax(pred, axis=1)
print("Predict that {} is: {}".format(sample_flower,species[pred]))

Predict that [[5.  3.  4.  2. ]
 [5.2 3.5 1.5 0.8]] is: ['Iris-versicolor' 'Iris-setosa']


# Load/Save Trained Network

Complex neural networks will take a long time to fit/train.  It is helpful to be able to save these neural networks so that they can be reloaded later.  A reloaded neural network will not require retraining.  Keras provides three formats for neural network saving.

* **YAML** - Stores the neural network structure (no weights) in the [YAML file format](https://en.wikipedia.org/wiki/YAML).
* **JSON** - Stores the neural network structure (no weights) in the [JSON file format](https://en.wikipedia.org/wiki/JSON).
* **HDF5** - Stores the complete neural network (with weights) in the [HDF5 file format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). 

Usually you will want to save in HDF5.

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
path = "./data/"
save_path = "./dnn/"

filename_read = os.path.join(path,"auto-mpg.csv")
df = pd.read_csv(filename_read,na_values=['NA','?'])

cars = df['name']
df.drop('name',1,inplace=True)
missing_median(df, 'horsepower')
x,y = to_xy(df,"mpg")
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x,y,verbose=0,epochs=100)

# Predict
pred = model.predict(x)

# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("Before save score (RMSE): {}".format(score))


# save entire network to HDF5 (save everything)
model.save(os.path.join(save_path,"network.hdf5"))

TypeError: DataFrame.drop() takes from 1 to 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

Now we reload the network and perform another prediction.  The RMSE should match the previous one exactly if the neural network was really saved and reloaded.

In [ ]:
from tensorflow.keras.models import load_model

model2 = load_model(os.path.join(save_path,"network.hdf5"))
pred = model2.predict(x)
# Measure RMSE error.  RMSE is common for regression.
score = np.sqrt(metrics.mean_squared_error(pred,y))
print("After load score (RMSE): {}".format(score))

After load score (RMSE): 4.3620195388793945


### References:

* [Google Colab](https://colab.research.google.com/) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow with free GPU support.  No setup needed.
* [IBM Cognitive Class Labs](https://www.datascientistworkbench.com) - Free web based platform that includes Python, Juypter Notebooks, and TensorFlow.  No setup needed.
* [Python Anaconda](https://www.continuum.io/downloads) - Python distribution that includes many data science packages, such as Numpy, Scipy, Scikit-Learn, Pandas, and much more.
* [TensorFlow](https://www.tensorflow.org/) - Google's mathematics package for deep learning.
* [Kaggle](https://www.kaggle.com/) - Competitive data science.  Good source of sample data.
* T81-558: Applications of Deep Neural Networks. Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/)